In [ ]:
import warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow info and warning messages
warnings.filterwarnings('ignore', category=DeprecationWarning)


Let’s start with something simple: the stack of two layers we used in the previous section. Its Functional API version looks like the following listing.

In [2]:
import keras
from keras import layers, models

#create an input layer (symbolic tensor)
inputs = keras.Input(shape=(3,), name="my_input")
#create a layer and call it with the input
features = layers.Dense(64, activation="relu")(inputs)

outputs = layers.Dense(10, activation="softmax")(features)

model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ my_input (InputLayer)           │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 906 (3.54 KB)

 Trainable params: 906 (3.54 KB)

 Non-trainable params: 0 (0.00 B)

the functional API is a way to create models that are more flexible than the Sequential API.

they hold more complex topologies, such as multi-input models and multi-output models.

example: 

Let’s say you’re building a system to rank customer support tickets by priority and route them to the appropriate department. Your model has three inputs:

- The title of the ticket (text)
- The text of the ticket (text)
- The priority of the ticket (categorical)

the model has two outputs:
- The priority of the ticket (categorical)
- The department to route the ticket to (categorical)

In [4]:
vocabulary_size = 10000
num_tags = 100
num_departments = 4

#define model inputs
title = keras.Input(shape=(vocabulary_size,), name="title")
text_body = keras.Input(shape=(vocabulary_size,), name="text_body")
tags = keras.Input(shape=(num_tags,), name="tags")

#combine input features into a single tensor
features = layers.Concatenate()([title, text_body, tags])
#apply a intermidiate layer to recombine input features into richer representation
features = layers.Dense(64, activation="relu")(features)

#define model outputs
priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
department = layers.Dense(
	num_departments, activation="softmax", name="department")(features)

model = keras.Model(inputs=[title, text_body, tags], outputs=[priority, department])

### TRAINING A MULTI-INPUT, MULTI-OUTPUT MODEL

In [6]:
import numpy as np

num_samples = 1280

title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))

priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

model.compile(
	optimizer="rmsprop",
	loss=["mean_squared_error", "categorical_crossentropy"],
	metrics=[["mean_absolute_error"], ["accuracy"]]
)

model.fit(
	[title_data, text_body_data, tags_data],
	[priority_data, department_data],
	epochs=3
)

model.evaluate([title_data, text_body_data, tags_data], [priority_data, department_data])

priority_preds, department_preds = model.predict([title_data, text_body_data, tags_data])

Epoch 1/3
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - department_accuracy: 0.2633 - department_loss: 63.4675 - loss: 63.7987 - priority_loss: 0.3312 - priority_mean_absolute_error: 0.4994
Epoch 2/3
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - department_accuracy: 0.2602 - department_loss: 64.6681 - loss: 64.9993 - priority_loss: 0.3312 - priority_mean_absolute_error: 0.4994
Epoch 3/3
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - department_accuracy: 0.2703 - department_loss: 71.5123 - loss: 71.8435 - priority_loss: 0.3312 - priority_mean_absolute_error: 0.4994
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - department_accuracy: 0.1352 - department_loss: 97.2811 - loss: 97.6123 - priority_loss: 0.3312 - priority_mean_absolute_error: 0.4994 
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


You can plot a Functional model as a graph with the plot_model()

In [7]:
import keras
import pydot
import graphviz
import tensorflow as tf

keras.utils.plot_model( model, "ticket_classifier_with_shape_info.png", show_shapes=True)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [8]:
model.layers

[<InputLayer name=title, built=True>,
 <InputLayer name=text_body, built=True>,
 <InputLayer name=tags, built=True>,
 <Concatenate name=concatenate_1, built=True>,
 <Dense name=dense_3, built=True>,
 <Dense name=priority, built=True>,
 <Dense name=department, built=True>]

In [9]:
model.layers[3].input

[<KerasTensor shape=(None, 10000), dtype=float32, sparse=False, ragged=False, name=title>,
 <KerasTensor shape=(None, 10000), dtype=float32, sparse=False, ragged=False, name=text_body>,
 <KerasTensor shape=(None, 100), dtype=float32, sparse=False, ragged=False, name=tags>]

In [10]:
model.layers[3].output

<KerasTensor shape=(None, 20100), dtype=float32, sparse=False, ragged=False, name=keras_tensor_6>

feature extraction, creating models that reuse intermediate features from another model.

Let’s say you want to add another output to the previous model—you want to estimate how long a given issue ticket will take to resolve, a kind of difficulty rating. You could do this via a classification layer over three categories: “quick,” “medium,” and “difficult.” You don’t need to recreate and retrain a model from scratch. You can start from the intermediate features of your previous model, since you have access to them, like this.

In [16]:
features = model.layers[4].output #layers[4] is the output of the previous model
difficulty = layers.Dense(3, activation="softmax", name="difficulty")(features)

new_model=keras.Model(
	inputs=[title, text_body, tags],
	outputs=[priority, department, difficulty]
)


You can train your model in much the same way as you would train a Sequential model, by calling fit() with lists of input and output data. These lists of data should be in the same order as the inputs you passed to the Model constructor.

In [17]:
keras.utils.plot_model( new_model, "updated_ticket_classifier.png", show_shapes=True)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


### Subclassing the Model class

The last model-building pattern you should know about is the most advanced one: Model subclassing. You learned in chapter 3 how to subclass the Layer class to create custom layers. Subclassing Model is pretty similar:

Subclassing Model is pretty similar: 
- In the __init__() method, define the layers the model will use.
- In the call() method, define the forward pass of the model, reusing the layers previously created. 
- Instantiate your subclass, and call it on data to create its weights.

REWRITING OUR PREVIOUS EXAMPLE AS A SUBCLASSED MODEL

In [ ]:
class CustomerTicketModel(keras.Model):
    def __init__(self, num_departments):
        super().__init__()
        self.concat_layer = layers.Concatenate()
        self.mixing_layer = layers.Dense(64, activation="relu")
        self.priority_scorer = layers.Dense(1, activation="sigmoid")
        self.department_classifier = layers.Dense(
            num_departments, activation="softmax"
        )

    #define forward pass in the call method
    def call(self, inputs):
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]

        features = self.concat_layer([title, text_body, tags])
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features) #output 1
        department = self.department_classifier(features) #output 2
        return priority, department


Once you’ve defined the model, you can instantiate it. Note that it will only create its weights the first time you call it on some data, much like Layer subclasses:

In [25]:
# Corrected CustomModel implementation
import tensorflow as tf

loss_fn = keras.losses.SparseCategoricalCrossentropy()
loss_tracker = keras.metrics.Mean(name="loss")

class CustomModel(keras.Model):
	def __init__(self, *args, **kwargs):
		super().__init__(*args, **kwargs)
		self.optimizer = None

	def train_step(self, data):
		inputs, targets = data
		with tf.GradientTape() as tape:
			predictions = self(inputs, training=True)
			loss = loss_fn(targets, predictions)
		gradients = tape.gradient(loss, self.trainable_weights)
		self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))

		loss_tracker.update_state(loss)
		return {"loss": loss_tracker.result()}

	def compile(self, optimizer, **kwargs):
		super().compile(**kwargs)
		self.optimizer = optimizer

	#lists metrics that should be reseted across epochs
	@property
	def metrics(self):
		return [loss_tracker]


In [26]:
# Corrected usage with proper compile parameters
inputs = keras.Input(shape=(28*28,))
features = layers.Dense(512, activation="relu")(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation="softmax")(features)
model = CustomModel(inputs, outputs)

# Complete compile with loss function and metrics
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Load MNIST data
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

# Train the model
model.fit(train_images, train_labels, epochs=3, validation_split=0.1)


Epoch 1/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.2900 - val_loss: 0.0000e+00
Epoch 2/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.1587 - val_loss: 0.0000e+00
Epoch 3/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.1279 - val_loss: 0.0000e+00


In [19]:
model = CustomerTicketModel(num_departments=4)

In [20]:
priority, department = model(
    {"title": title_data, "text_body": text_body_data, "tags": tags_data}
)

What, then, is the difference between a Layer subclass and a Model subclass? It’s simple: a “layer” is a building block you use to create models, and a “model” is the top-level object that you will actually train, export for inference, etc. In short, a Model has fit(), evaluate(), and predict() methods. Layers don’t. Other than that, the two classes are virtually identical. (Another difference is that you can save a model to a file on disk)

You can compile and train a Model subclass just like a Sequential or Functional model:

In [21]:
model.compile(optimizer="rmsprop",
    loss=["mean_squared_error", "categorical_crossentropy"],
    metrics=[["mean_absolute_error"], ["accuracy"]])

model.fit({"title": title_data,
        "text_body": text_body_data,
        "tags":tags_data},
        [priority_data, department_data],
        epochs=1)

model.evaluate({"title": title_data,
                "text_body": text_body_data,
                "tags":tags_data},
                [priority_data, department_data])

priority_preds, department_preds = model.predict(
{"title": title_data, "text_body": text_body_data, "tags": tags_data})



40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.2937 - categorical_crossentropy_loss: 31.1573 - loss: 31.4721 - mean_absolute_error: 0.4808 - mean_squared_error_loss: 0.3148
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5664 - categorical_crossentropy_loss: 35.3319 - loss: 35.6525 - mean_absolute_error: 0.4873 - mean_squared_error_loss: 0.3206
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


The Model subclassing workflow is the most flexible way to build a model. It enables you to build models that cannot be expressed as directed acyclic graphs of layersimagine, for instance, a model where the call() method uses layers inside a for loop, or even calls them recursively. Anything is possible—you’re in charge.

Functional and subclassed models are also substantially different in nature. A Functional model is an explicit data structure—a graph of layers, which you can view, inspect, and modify. A subclassed model is a piece of bytecode—a Python class with a call() method that contains raw code. This is the source of the subclassing workflow’s flexibility—you can code up whatever functionality you like—but it introduces new limitations. For instance, because the way layers are connected to each other is hidden inside the body of the call() method, you cannot access that information. Calling summary() will not display layer connectivity, and you cannot plot the model topology via plot_model(). Likewise, if you have a subclassed model, you cannot access the nodes of the graph of layers to do feature extraction because there is simply no graph. Once the model is instantiated, its forward pass becomes a complete black box.

### 7.2.4 Mixing and matching different components

you can use a subclassed layer or model in a Functional model.

In [ ]:
class Classifier(keras.Model):
    def __init__(self, num_classes=2):
        super().__init__()
        #se é classficiacao binaria ou multiclasse
        if num_classes ==2:
            num_units = 1
            activation = "sigmoid"
        else:
            num_units = num_classes
            activation = "softmax"

        self.dense = layers.Dense(num_units, activation=activation)
    def call(self, inputs):
        return self.dense(inputs)

inputs = keras.Input(shape=(3,))
features = layers.Dense(64, activation="relu") (inputs)
outputs = Classifier(num_classes=10) (features)
model = keras.Model(inputs=inputs, outputs=outputs)

Inversely, you can use a Functional model as part of a subclassed layer or model

In [ ]:
inputs = keras.Input(shape=(64,))
outputs = layers.Dense(1, activation="sigmoid")(inputs)
binary_classifier = keras.Model(inputs=inputs, outputs=outputs)

class MyModel(keras.Model):
    def __init__(self, num_classes=2):
        super().__init__()
        self.dense = layers.Dense(64, activation="relu")
        self.classifier = binary_classifier

    def call(self, inputs):
        features = self.dense(inputs)
        return self.classifier(features)
model = MyModel()

### 7.3 Using built-in training and evaluation loops

In [2]:
import keras
from keras import layers
from keras.datasets import mnist

def get_mnist_model():
    inputs = keras.Input(shape=(28*28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5) (features)
    outputs = layers.Dense(10, activation="softmax") (features)

    model = keras.Model(inputs, outputs)
    return model

(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28*28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28*28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

model = get_mnist_model()
model.compile(optimizer="rmsprop",
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
model.fit(train_images, train_labels,
        epochs=3,
        validation_data=(val_images,val_labels))
#compute loss and metrics on new data
test_metrics = model.evaluate(test_images, test_labels)
#compute classification probabilities on new data
predictions = model.predict(test_images)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9123 - loss: 0.2951 - val_accuracy: 0.9564 - val_loss: 0.1502
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9538 - loss: 0.1584 - val_accuracy: 0.9693 - val_loss: 0.1121
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9628 - loss: 0.1314 - val_accuracy: 0.9715 - val_loss: 0.1044
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9729 - loss: 0.0931  
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 891us/step


There are a couple of ways you can customize this simple workflow: 
- Provide your own custom metrics.
- Pass callbacks to the fit() method to schedule actions to be taken at specific points during training.

### 7.3.1 Writing your own metrics

In [ ]:
import tensorflow as tf

class RootMeanSquaredError(keras.metrics.Metric):
    #Define the state  variables in the  constructor.
    #Like for layers, you  have access to  the add_weight() method.
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32"
        )

    #implement the state update logic in update_state(). the y_true argument
    #is the targets for one batch, while y_pred represents the
    # corresponding predicitons from the model.
    # sample_weights wont be used here
    def update_state(self, y_true, y_pred, sample_weight=None):
    #to match our MNIST modal,we expect categorical predicitons and integer labels
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred) [0]
        self.total_samples.assign_add(num_samples)
    #You use the result() method to return the current value of the metric:
    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

    #Meanwhile, you also need to expose a way
    #  to reset the metric state without
    #  having to reinstantiate it—this
    #  enables the same metric objects to be
    #  used across different epochs of training
    # or across both training and evaluation.
    def reset_state(self):
        self.mse_sum.assign(0.)
        self.total_samples.assign(0)

Custom metrics can be used just like built-in ones. Let’s test-drive our own metric:

In [ ]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy", RootMeanSquaredError()])

model.fit(train_images, train_labels,
        epochs=3,
        validation_data=(val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9115 - loss: 0.2984 - rmse: 0.3652 - val_accuracy: 0.9590 - val_loss: 0.1416 - val_rmse: 0.2493
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9540 - loss: 0.1625 - rmse: 0.2666 - val_accuracy: 0.9667 - val_loss: 0.1159 - val_rmse: 0.2254
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9632 - loss: 0.1302 - rmse: 0.2393 - val_accuracy: 0.9689 - val_loss: 0.1095 - val_rmse: 0.2152
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9707 - loss: 0.1067 - rmse: 0.2133


### 7.3.2 Using callbacks

A callback is an object (a class instance implementing specific methods) that is passed to the model in the call to fit() and that is called by the model at various points during training. It has access to all the available data about the state of the model and its performance, and it can take action: interrupt training, save a model, load a different weight set, or otherwise alter the state of the model.

callback examples:
- Model checkpointing—Saving the current state of the model at different points during training.
- Early stopping—Interrupting training when the model’s performance on a validation set stops improving.
- Dynamically adjusting the value of certain parameters during training —Such as the learning rate of the optimizer.
- Logging training and validation metrics during training, or visualizing the representations learned by the model as they’re updated—The fit() progress bar that you’re familiar with is in fact a callback!


### THE EARLYSTOPPING AND MODELCHECKPOINT CALLBACKS

In [4]:
import keras
callbacks_list = [
	keras.callbacks.EarlyStopping(
		monitor="val_accuracy",
		patience=2,
	),
	#saves the current weights after every epoch
	keras.callbacks.ModelCheckpoint(
		filepath="checkpoint_path.keras",
		monitor="val_accuracy",#These two arguments mean you wont  overwrite the model
		# file unless val_loss  has improved, which allows you to keep  the best model seen during training.
		save_best_only=True,
	)
]

model = get_mnist_model()
model.compile(optimizer="rmsprop",
			loss="sparse_categorical_crossentropy",
			metrics=["accuracy"])
model.fit(train_images, train_labels,
		epochs=10,
		callbacks=callbacks_list,
		validation_data=(val_images, val_labels))



Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9107 - loss: 0.2961 - val_accuracy: 0.9546 - val_loss: 0.1558
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9539 - loss: 0.1586 - val_accuracy: 0.9683 - val_loss: 0.1121
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9624 - loss: 0.1315 - val_accuracy: 0.9723 - val_loss: 0.1054
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9676 - loss: 0.1160 - val_accuracy: 0.9741 - val_loss: 0.0989
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9719 - loss: 0.1041 - val_accuracy: 0.9736 - val_loss: 0.1013
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9742 - loss: 0.0939 - val_accuracy: 0.9773 - val_loss: 0.0965
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9757 - loss: 0.0876 - val_accuracy: 0.9745 - val_loss: 0.1049
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9775 - loss: 0.0825 - 

### 7.3.3 Writing your own callbacks

Callbacks are implemented by subclassing the keras.callbacks.Callback class. You can then implement any number of the following transparently named methods, which are called at various points during training:
- on_epoch_begin
- on_epoch_end
- on_batch_begin
- on_batch_end
- on_train_begin
- on_train_end

These methods are all called with a logs argument, which is a dictionary containing information about the previous batch, epoch, or training run—training and validation metrics, and so on. The on_epoch_* and on_batch_* methods also take the epoch or batch index as their first argument (an integer). Here’s a simple example that saves a list of per-batch loss values during training and saves a graph of these values at the end of each epoch.

In [10]:
from matplotlib import pyplot as plt
#example that saves a list of per-batch loss values during training
#  and saves a graph of these values at the end of each epoch
class LossHistory(keras.callbacks.Callback):

	def on_train_begin(self, logs):
		self.per_batch_losses = []

	def on_batch_end(self, batch, logs):
		self.per_batch_losses.append(logs.get("loss"))

	def on_epoch_end(self, epoch, logs):
		plt.clf()
		plt.plot(range(len(self.per_batch_losses)), self.per_batch_losses,
				label="Training loss for each batch")
		plt.xlabel(f"Batch (epoch {epoch})")
		plt.ylabel("Loss")
		plt.legend()
		plt.savefig(f"plot_at_epoch_{epoch}.png")
		plt.close()

In [11]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
			loss="sparse_categorical_crossentropy",
			metrics=["accuracy"])
model.fit(train_images, train_labels,
		epochs=10,
		callbacks=[LossHistory()],
		validation_data=(val_images, val_labels))

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9126 - loss: 0.2938 - val_accuracy: 0.9573 - val_loss: 0.1478
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9536 - loss: 0.1589 - val_accuracy: 0.9667 - val_loss: 0.1140
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9633 - loss: 0.1308 - val_accuracy: 0.9722 - val_loss: 0.0997
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9681 - loss: 0.1142 - val_accuracy: 0.9741 - val_loss: 0.0992
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9717 - loss: 0.1044 - val_accuracy: 0.9765 - val_loss: 0.0937
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9746 - loss: 0.0926 - val_accuracy: 0.9784 - val_loss: 0.0883
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9763 - loss: 0.0863 - val_accuracy: 0.9789 - val_loss: 0.0917
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9775 - loss: 0.0829 - 

### 7.3.4 Monitoring and visualization with TensorBoard

In [3]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
			loss="sparse_categorical_crossentropy",
			metrics=["accuracy"])

tensorboard = keras.callbacks.TensorBoard(
		log_dir="logs"
)

model.fit(train_images, train_labels,
		epochs=10,
		validation_data=(val_images, val_labels),
		callbacks=[tensorboard])


%load_ext tensorboard
%tensorboard --logdir=./logs


Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9122 - loss: 0.2974 - val_accuracy: 0.9545 - val_loss: 0.1517
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9535 - loss: 0.1598 - val_accuracy: 0.9692 - val_loss: 0.1132
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9634 - loss: 0.1325 - val_accuracy: 0.9723 - val_loss: 0.1083
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9683 - loss: 0.1124 - val_accuracy: 0.9766 - val_loss: 0.0998
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9715 - loss: 0.1028 - val_accuracy: 0.9765 - val_loss: 0.0952
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9739 - loss: 0.0939 - val_accuracy: 0.9768 - val_loss: 0.0936
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9754 - loss: 0.0905 - val_accuracy: 0.9755 - val_loss: 0.1009
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9781 - loss: 0.0830 - 

Reusing TensorBoard on port 6006 (pid 20632), started 0:20:52 ago. (Use '!kill 20632' to kill it.)

Once the model starts running, it will write logs at the target location. If you are running your Python script on a local machine, you can then launch the local TensorBoard server using the following command:  

tensorboard --logdir /full_path_to_your_log_dir

### 7.4 Writing your own training and evaluation loops

### 7.4.1 Training versus inference

Some Keras layers, such as the Dropout layer, have different behaviors during training and during inference (when you use them to generate predictions). Such layers expose a training Boolean argument in their call() method. Calling dropout(inputs, training=True) will drop some activation entries, while calling dropout(inputs, training=False) does nothing. By extension, Functional and Sequential models also expose this training argument in their call() methods. Remember to pass training =True when you call a Keras model during the forward pass! Our forward pass thus becomes predictions = model(inputs, training=True).

In addition, note that when you retrieve the gradients of the weights of your model, you should not use tape.gradients(loss, model.weights), but rather tape .gradients(loss, model.trainable_weights). Indeed, layers and models own two kinds of weights:

- Trainable weights—These are meant to be updated via backpropagation to minimize the loss of the model, such as the kernel and bias of a Dense layer.
- Non-trainable weights—These are meant to be updated during the forward pass by the layers that own them. For instance, if you wanted a custom layer to keep a counter of how many batches it has processed so far, that information would be stored in a non-trainable weight, and at each batch, your layer would increment the counter by one.

In [ ]:
def train_step(inputs, targets):
	with tf.GradientTape() as tape:
		predictions = model(inputs, training=True)
		loss = loss_fn(targets, predictions)
		gradients = tape.gradients(loss, model.trainable_weights)
		optimizer.apply_gradients(zip(model.trainable_weights, gradients))

### 7.4.2 Low-level usage of metrics

for each batch of targets and predictions, and then use result() to query the current metric value:

In [4]:
metric = keras.metrics.SparseCategoricalAccuracy()
targets = [0, 1, 2]
predictions = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
metric.update_state(targets, predictions)
result = metric.result()
print(f"result: {result.numpy()}")




result: 1.0


you may also need to track the average of a scalar value, such as the model’s loss. You can do this via the keras.metrics.Mean metric:

In [5]:
values = [0, 1, 2, 3, 4]
mean_tracker = keras.metrics.Mean()
for value in values:
	mean_tracker.update_state(value)
print(f"Mean of values: {mean_tracker.result().numpy()}")

Mean of values: 2.0


7.4.3 A complete training and evaluation loop

Let’s combine the forward pass, backward pass, and metrics tracking into a fit()-like training step function that takes a batch of data and targets and returns the logs that would get displayed by the fit() progress bar.

In [17]:
import tensorflow as tf
model = get_mnist_model()

loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.RMSprop()
metrics = [keras.metrics.SparseCategoricalAccuracy()]
loss_tracking_metric = keras.metrics.Mean()

def train_step(inputs, targets):
	with tf.GradientTape() as tape:
		predictions = model(inputs, training=True)
		loss = loss_fn(targets, predictions)
	gradients = tape.gradient(loss, model.trainable_weights)
	optimizer.apply_gradients(zip(gradients, model.trainable_weights))
	logs = {}
	for metric in metrics:
		metric.update_state(targets, predictions)
		logs[metric.name] = metric.result()
	loss_tracking_metric.update_state(loss)
	logs["loss"] = loss_tracking_metric.result()
	return logs



We will need to reset the state of our metrics at the start of each epoch and before running evaluation. Here’s a utility function to do it:

In [19]:
def reset_metrics():
	for metric in metrics:
		metric.reset_state()
	loss_tracking_metric.reset_state()

In [20]:
training_dataset = tf.data.Dataset.from_tensor_slices(
	(train_images, train_labels))
training_dataset = training_dataset.batch(32)

for epoch in range(3):
	reset_metrics()
	for inputs_batch, targets_batch in training_dataset:
		logs = train_step(inputs_batch, targets_batch)
	print(f"Epoch {epoch}: {logs}")
	for key, value in logs.items():
		print(f"{key}: {value:.4f}")






Epoch 0: {'sparse_categorical_accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.9146199822425842>, 'loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.29007312655448914>}
sparse_categorical_accuracy: 0.9146
loss: 0.2901


KeyboardInterrupt: 

And here’s the evaluation loop: a simple for loop that repeatedly calls a test_step() function, which processes a single batch of data. The test_step() function is just a subset of the logic of train_step()

In [22]:
def test_step(inputs, targets):
	predictions = model(inputs, training=False)
	loss = loss_fn(targets, predictions)
	logs = {}
	for metric in metrics:
		metric.update_state(targets, predictions)
		logs["val_" + metric.name] = metric.result()
	loss_tracking_metric.update_state(loss)
	logs["val_loss"] = loss_tracking_metric.result()
	return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()
for inputs_batch, targets_batch in val_dataset:
	logs = test_step(inputs_batch, targets_batch)
print("Evaluation results:")
for key, value in logs.items():
	print(f"{key}: {value:.4f}")



Evaluation results:
val_sparse_categorical_accuracy: 0.0596
val_loss: 2.3579


Congrats—you’ve just reimplemented fit() and evaluate()! Or almost: fit() and evaluate() support many more features, including large-scale distributed computation, which requires a bit more work. It also includes several key performance optimizations.

### 7.4.4 Make it fast with tf.function

In [14]:
@tf.function
def test_step(inputs, targets):
	predictions = model(inputs, training=False)
	loss = loss_fn(targets, predictions)

	logs = {}
	for metric in metrics:
		metric.update_state(targets, predictions)
		logs["val_" + metric.name] = metric.result()
	loss_tracking_metric.update_state(loss)
	logs["val_loss"] = loss_tracking_metric.result()
	return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()
for inputs_batch, targets_batch in val_dataset:
	logs = test_step(inputs_batch, targets_batch)
print("Evaluation results:")
for key, value in logs.items():
	print(f"{key}: {value:.4f}")



Evaluation results:
val_sparse_categorical_accuracy: 0.9684
val_loss: 0.1181


@tf.function é aconselhavel quando o modelo já está otimizado e não precisa ser mais alterado.

### 7.4.5 leveraging fit() with a custom training loop

- We create a new class that subclasses keras.Model.
- We override the method train_step(self, data). Its contents are nearly identical to what we used in the previous section. It returns a dictionary mapping metric names (including the loss) to their current values.
- We implement a metrics property that tracks the model’s Metric instances. This enables the model to automatically call reset_state() on the model’s metrics at the start of each epoch and at the start of a call to evaluate(), so you don’t have to do it by hand.

In [30]:
import tensorflow as tf  # Add this import

loss_fn = keras.losses.SparseCategoricalCrossentropy()
loss_tracker = keras.metrics.Mean(name="loss")

class CustomModel(keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.optimizer = None  # Store optimizer here

    def train_step(self, data):
        inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = loss_fn(targets, predictions)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))  # Use self.optimizer

        loss_tracker.update_state(loss)
        return {"loss": loss_tracker.result()}

    def compile(self, optimizer, **kwargs):  # Add custom compile method
        super().compile(**kwargs)
        self.optimizer = optimizer  # Store the optimizer

    #lists metrics that should be reseted across epochs
    @property
    def metrics(self):
        return [loss_tracker]

In [31]:
inputs = keras.Input(shape=(28*28,))
features = layers.Dense(512, activation="relu")(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation="softmax") (features)
model = CustomModel(inputs, outputs)

model.compile(optimizer=keras.optimizers.RMSprop())
model.fit(train_images, train_labels, epochs=3)

Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.2779
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.1496
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.1250


- This pattern does not prevent you from building models with the Functional API. You can do this whether you’re building Sequential models, Functional API models, or subclassed models.
- You don’t need to use a @tf.function decorator when you override train_ step—the framework does it for you.

Now, what about metrics, and what about configuring the loss via compile()? After you’ve called compile(), you get access to the following

- self.compiled_loss—The loss function you passed to compile().
-  self.compiled_metrics—A wrapper for the list of metrics you passed, which allows you to call self.compiled_metrics.update_state() to update all of your metrics at once.
- self.metrics—The actual list of metrics you passed to compile(). Note that it also includes a metric that tracks the loss, similar to what we did manually with our loss_tracking_metric earlier.

In [32]:
class CustomModel(keras.Model):
	def train_step(self, data):
		inputs, targets = data
		with tf.GradientTape() as tape:
			predictions = self(inputs, training=True)
			loss = self.compiled_loss(targets, predictions)
		gradients = tape.gradient(loss, self.trainable_weights)
		self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
		#update models metrics
		self.compiled_metrics.update_state(targets, predictions)
		#return metric values in a dictionary
		return {m.name: m.result() for m in self.metrics}

In [33]:
inputs = keras.Input(shape=(28*28,))
features = layers.Dense(512, activation="relu")(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation="softmax") (features)
model = CustomModel(inputs, outputs)

model.compile(optimizer=keras.optimizers.RMSprop(),
			loss=keras.losses.SparseCategoricalCrossentropy(),
			metrics=[keras.metrics.SparseCategoricalAccuracy()])
model.fit(train_images, train_labels, epochs=3)






Epoch 1/3


c:\Users\ryanz\estudo-ufal\dl\DeepLearningWithPython\env\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:673: UserWarning: `model.compiled_loss()` is deprecated. Instead, use `model.compute_loss(x, y, y_pred, sample_weight, training)`.
  warnings.warn(
c:\Users\ryanz\estudo-ufal\dl\DeepLearningWithPython\env\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:648: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - sparse_categorical_accuracy: 0.9184 - loss: 0.1000
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - sparse_categorical_accuracy: 0.9556 - loss: 0.1000
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - sparse_categorical_accuracy: 0.9641 - loss: 0.1000


## sumary

- Keras offers a spectrum of different workflows, based on the principle of progressive disclosure of complexity. They all smoothly inter-operate together.
- You can build models via the Sequential class, via the Functional API, or by subclassing the Model class. Most of the time, you’ll be using the Functional API.
- The simplest way to train and evaluate a model is via the default fit() and evaluate() methods.
-  Keras callbacks provide a simple way to monitor models during your call to fit() and automatically take action based on the state of the model.
- You can also fully take control of what fit() does by overriding the train_ step() method.
- Beyond fit(), you can also write your own training loops entirely from scratch. This is useful for researchers implementing brand-new training algorithms.